In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["PYTORCH_USE_CUDA_DSA"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [ ]:
%pip install transformers
#!pip install transformers==4.30

###Fine Tuning

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


Tutorial source: https://huggingface.co/docs/transformers/en/training

In [ ]:
%pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install transformers==4.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/marcoguerini/CONAN/master/DIALOCONAN/DIALOCONAN.csv")
df = df.drop(columns=['TARGET', 'dialogue_id', 'turn_id', 'source'])
df.loc[df['type'] == "CN", 'type'] = 1
df.loc[df['type'] == "HS", 'type'] = 0
df = df.rename(columns={'type': 'label'}).reset_index(drop=True)

In [ ]:
#https://github.com/VinAIResearch/BERTweet

from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased", do_lower_case= False, max_len= 512, unk_token= "[UNK]", sep_token= "[SEP]", pad_token= "[PAD]", cls_token= "[CLS]", mask_token= "[MASK]")
tokenizer = AutoTokenizer.from_pretrained(
  cls_token= "<s>",
  eos_token= "</s>",
  mask_token= "<mask>",
  model_max_length= 128,
  pretrained_model_name_or_path= "vinai/bertweet-base",
  normalization= False,
  pad_token= "<pad>",
  sep_token= "</s>",
  tokenizer_class= "BertweetTokenizer",
  unk_token= "<unk>")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=120)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
all_data = Dataset.from_pandas(df)
all_data_tok = all_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/16625 [00:00<?, ? examples/s]

In [ ]:
# 70 train, 10, val, 20 test
train_other = all_data_tok.train_test_split(test_size=0.3)
val_test = train_other['test'].train_test_split(test_size=2/3)
combined_dataset = DatasetDict({'train': train_other['train'],'val': val_test['train'],'test': val_test['test']})

In [ ]:
small_train_data_tok = combined_dataset["train"]
small_val_data_tok = combined_dataset["val"]
small_test_data_tok = combined_dataset["test"]

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="training_checkpoints",
    evaluation_strategy="no",
    learning_rate=0.00004,
    num_train_epochs=3
)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
%pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
%cd "/content/drive/MyDrive/CMU/Spring 2024/05318/Project/"

Mounted at /content/drive/
/content/drive/MyDrive/CMU/Spring 2024/05318/Project


In [ ]:
df = pd.read_csv("twitter_sentiment.csv")
df = df.drop(columns=['id'])
import numpy as np

not_hate = df.loc[df['label'] == 0]
not_hate = not_hate.loc[np.random.choice(not_hate.index, 1000, replace=False)]

hate = df.loc[df['label'] == 1]
hate = hate.loc[np.random.choice(hate.index, 1000, replace=False)]

df = pd.concat([hate, not_hate])


df = df.rename(columns={'tweet': 'text'}).reset_index(drop=True)
twitter_data = Dataset.from_pandas(df)
twitter_data_tok = twitter_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
def conan_evaluate(model):
  trainer = Trainer(
    model=model
  )
  conan_results = trainer.predict(test_dataset=small_test_data_tok)
  conan_true_labels = np.asarray(combined_dataset["test"]['label'][:len(small_test_data_tok)])
  conan_logits = conan_results[0]
  conan_predictions = np.argmax(conan_logits, axis=-1)

  true_positive = np.sum(np.logical_and(conan_predictions == 1, conan_true_labels == 1))

  false_positive = np.sum(np.logical_and(conan_predictions == 1, conan_true_labels == 0))

  true_negative = np.sum(np.logical_and(conan_predictions == 0, conan_true_labels == 0))

  false_negative = np.sum(np.logical_and(conan_predictions == 0, conan_true_labels == 1))

  return [true_positive, false_positive, true_negative, false_negative]

In [ ]:
def twitter_evaluate(model):
  trainer = Trainer(
    model=model
  )
  twitter_results = trainer.predict(test_dataset=twitter_data_tok)
  twitter_true_labels = np.asarray(twitter_data['label'])
  twitter_logits = twitter_results[0]
  twitter_predictions = np.argmax(twitter_logits, axis=-1)

  true_positive = np.sum(np.logical_and(twitter_predictions == 1, twitter_true_labels == 1))

  false_positive = np.sum(np.logical_and(twitter_predictions == 1, twitter_true_labels == 0))

  true_negative = np.sum(np.logical_and(twitter_predictions == 0, twitter_true_labels == 0))

  false_negative = np.sum(np.logical_and(twitter_predictions == 0, twitter_true_labels == 1))

  return [true_positive, false_positive, true_negative, false_negative]



In [ ]:
def run_hyperparameter_test(epochs, lr, log_df):

  model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

  training_args = TrainingArguments(
  output_dir="training_checkpoints",
  learning_rate=lr,
  num_train_epochs=epochs
  )

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_data_tok,
  )
  trainer.train()

  model.save_pretrained(f"""fullData_hateSpeech_lr{lr}_epochs{epochs}_3""", from_pt=True)

  conan_eval = conan_evaluate(model)
  twitter_eval = twitter_evaluate(model)
  res = [lr, epochs] + conan_eval + twitter_eval
  print(res)

  log_df.loc[len(log_df.index)] = res
  return log_df



In [ ]:
from tqdm import tqdm

%cd "/content/drive/MyDrive/CMU/Spring 2024/05318/Project/"

def run_all_hyperparameters():
  log_df = pd.DataFrame(columns=['lr','epochs','conan_tp','conan_fp','conan_tn','conan_fn','twit_tp', 'twit_fp', 'twit_tn', 'twit_fn'])
  for epochs in tqdm([1,2,3]):
    for lr in tqdm([1e-10, 1e-7, 1e-5, 1e-4, 1e-3]):
      log_df = run_hyperparameter_test(epochs, lr, log_df)
      log_df.to_csv("hyperparameters_testing_3_log.csv")

run_all_hyperparameters()

/content/drive/MyDrive/CMU/Spring 2024/05318/Project


  0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

[0.0001, 3, 1579, 59, 1646, 42, 369, 533, 467, 631]


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.734900
1000,0.748800
1500,0.734600
2000,0.713700
2500,0.707500
3000,0.703300
3500,0.697200
4000,0.696300



100%|██████████| 3/3 [2:15:30<00:00, 2710.13s/it]

[0.001, 3, 0, 0, 1705, 1621, 0, 0, 1000, 1000]
